In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
!pip install -q transformers datasets accelerate

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset

2025-06-09 10:00:41.792696: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749463241.817213     189 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749463241.824831     189 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
dataset = load_dataset("FiscalNote/billsum")

In [6]:
train_data = dataset["train"]
test_data = dataset["test"]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})

In [8]:
train_data

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 18949
})

In [9]:
test_data

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 3269
})

In [10]:
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [11]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["text", "summary", "title"])


Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [13]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-p100-output",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    predict_with_generate=True,
    # fp16=False for P100 (no native support)
    gradient_checkpointing=True,
    generation_max_length=128,
    logging_dir="./logs",
    logging_steps=250,
    report_to="none",  # or "wandb"
)


In [14]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipykernel_189/2112273467.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [15]:
!nvidia-smi

Mon Jun  9 10:02:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             33W /  250W |    1271MiB /  16384MiB |     25%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
import time
start = time.time()
trainer.train()
print("Training finished in", time.time() - start, "seconds.")


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,1.468100,1.310732
2,1.303800,1.262789


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Training finished in 10901.266356229782 seconds.


In [17]:
trainer.save_model("./flan-t5-legal-summary")
tokenizer.save_pretrained("./flan-t5-legal-summary")


('./flan-t5-legal-summary/tokenizer_config.json',
 './flan-t5-legal-summary/special_tokens_map.json',
 './flan-t5-legal-summary/spiece.model',
 './flan-t5-legal-summary/added_tokens.json',
 './flan-t5-legal-summary/tokenizer.json')

In [20]:
from transformers import pipeline

summarizer = pipeline("summarization", model="./flan-t5-legal-summary", tokenizer="./flan-t5-legal-summary")

sample_text = dataset["test"][0]["text"]
summary = summarizer("summarize: " + sample_text, max_length=128, truncation=True)[0]['summary_text']

print("🔍 Original Title:", dataset["test"][0]["title"])
print("📝 Generated Summary:", summary)
print("📄 Reference Summary:", dataset["test"][0]["summary"])


Device set to use cuda:0


🔍 Original Title: To make technical corrections to the Water Resources Development Act of 1999.
📝 Generated Summary: Amends the Water Resources Development Act of 1992 to provide for the provision of an alternative water supply and a project for the elimination or control of combined sewer overflows for Jackson County, Mississippi, Manchester, New Hampshire, Atlanta, Georgia, Paterson, Passaic County, New Jersey, Elizabeth and North Hudson, New York, and New Jersey. Requires the North Hudson Sewerage Authority to carry out the Upper Mississippi River Environmental Management Program. Authorizes the Secretary of the Interior to enter into a cooperative agreement with the United States Environmental Protection Agency (USAEPA) to develop and implement a
📄 Reference Summary: Amends the Water Resources Development Act of 1999 to: (1) authorize appropriations for FY 1999 through 2009 for implementation of a long-term resource monitoring program with respect to the Upper Mississippi River Env

In [27]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

# ✅ Load your fine-tuned summarization model
summ_model_path = "./flan-t5-legal-summary"  # or use absolute path if needed

summ_tokenizer = AutoTokenizer.from_pretrained(summ_model_path)
summ_model = AutoModelForSeq2SeqLM.from_pretrained(summ_model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summ_model.to(device)

# ✅ Load a pretrained NER model
# You can replace this with a legal-specific NER model if available
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple", device=0 if torch.cuda.is_available() else -1)

# ✅ Inference function
def process_legal_document(text):
    # ---- Summarization ----
    inputs = summ_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=1024).to(device)
    summary_ids = summ_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=200,
        num_beams=4,
        early_stopping=True
    )
    summary = summ_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # ---- NER ----
    ner_results = ner_pipeline(text)

    return summary, ner_results

# ✅ Example usage
if __name__ == "__main__":
    legal_text = """
    IN THE SUPREME COURT OF PAKISTAN  
CIVIL APPELLATE JURISDICTION  
CIVIL APPEAL NO. 1456 OF 2022  

Between:  
Mr. Ahsan Raza, Resident of F-6/3, Islamabad  
— Appellant  

And  

The Federal Board of Revenue (FBR), through Chairman, Islamabad  
The Ministry of Law and Justice, Government of Pakistan  
The Commissioner Inland Revenue, Zone-IV, Karachi  
— Respondents  

JUDGMENT

Justice Syed Mansoor Ali Shah delivering the opinion of the Court:

The present appeal arises out of the impugned judgment dated 16th November 2021 passed by the Islamabad High Court in W.P. No. 3189/2020, whereby the petitioner’s plea for tax exemption under Section 53(2)(c) of the Income Tax Ordinance, 2001 was dismissed.

The appellant, Mr. Ahsan Raza, a corporate consultant by profession, claims that the consultancy fee received by him from GlobalTech Solutions (Pvt.) Ltd. — a Singapore-based IT firm with regional operations in Pakistan — was wrongly taxed by the Inland Revenue Department under Section 21(b)(iv) of the Ordinance. He contends that the said fee qualifies as income from a foreign source and should be exempt under bilateral tax treaties signed between Pakistan and the Republic of Singapore.

The FBR, on the other hand, asserts that the consultancy was rendered within Pakistan's territorial jurisdiction and thus attracts domestic tax obligations. The Ministry of Law and Justice also submitted that no overriding clause in the treaty nullifies the domestic provisions in this context.

After hearing both parties and examining the nature of cross-border service agreements submitted into evidence — including the Memorandum of Understanding (MoU) signed on 12th February 2020 between Mr. Raza and GlobalTech — the Court is of the opinion that the payment was subject to tax under Pakistani law.

Accordingly, the appeal is dismissed. No order as to costs.

    """

    summary, entities = process_legal_document(legal_text)

    print("📄 Summary:\n", summary)
    print("\n🧾 Named Entities:")
    for ent in entities:
        print(f"{ent['word']} ({ent['entity_group']}) — Score: {ent['score']:.2f}")


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


📄 Summary:
 (This measure has not been amended since it was passed by the Islamabad High Court in W.P. No. 3189/2020. The summary of that version is repeated here.)

🧾 Named Entities:
PA (LOC) — Score: 1.00
##K (ORG) — Score: 0.51
##IS (LOC) — Score: 1.00
##TA (ORG) — Score: 0.61
##N (LOC) — Score: 0.94
Ah (PER) — Score: 1.00
##san Raza (PER) — Score: 0.87
Islamabad (LOC) — Score: 1.00
Federal Board of Revenue (ORG) — Score: 0.99
FBR (ORG) — Score: 0.96
Islamabad (LOC) — Score: 1.00
Ministry of Law and Justice (ORG) — Score: 1.00
Government of Pakistan (ORG) — Score: 0.99
Inland Revenue (ORG) — Score: 0.72
Karachi (LOC) — Score: 0.97
Re (ORG) — Score: 0.56
Syed Mansoor Ali Shah (PER) — Score: 0.99
Islam (ORG) — Score: 0.74
##abad (LOC) — Score: 0.51
High Court (ORG) — Score: 0.94
P (ORG) — Score: 0.48
Tax Ordinance (MISC) — Score: 0.90
Ahsan Raza (PER) — Score: 0.95
GlobalTech Solutions (ORG) — Score: 1.00
Pvt. ) Ltd (ORG) — Score: 0.89
Singapore (MISC) — Score: 0.96
Pakistan (LOC) — S

In [26]:
!zip -r flan-t5-legal-summary.zip flan-t5-legal-summary


updating: flan-t5-legal-summary/ (stored 0%)
updating: flan-t5-legal-summary/training_args.bin (deflated 52%)
updating: flan-t5-legal-summary/spiece.model (deflated 48%)
updating: flan-t5-legal-summary/tokenizer.json (deflated 74%)
updating: flan-t5-legal-summary/generation_config.json (deflated 29%)
updating: flan-t5-legal-summary/special_tokens_map.json (deflated 85%)
updating: flan-t5-legal-summary/tokenizer_config.json (deflated 95%)
updating: flan-t5-legal-summary/config.json (deflated 62%)
updating: flan-t5-legal-summary/model.safetensors (deflated 7%)
